In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
interesting = """242_Yahoo_A2synthetic_66_data_500_893                             -1.0
678_TODS_12_500_26                                                -0.8
784_OPPORTUNITY_S4-ADL5_1100_1200_col_id_17_1100_1200             -0.8
780_OPPORTUNITY_S2-ADL3_7295_27516_col_id_6_7295_27516            -0.8
789_OPPORTUNITY_S3-ADL5_1477_1577_col_id_5_1477_1577              -0.8
256_Yahoo_A4Benchmark-TS66_data_500_202                            0.5
688_Exathlon_5_1_100000_63-33_10766_12590_col_id_27_10766_12590    0.6
162_UCR_Anomaly_DISTORTEDInternalBleeding17_1875_3198              0.6
267_Yahoo_A3Benchmark-TS94_data_500_577                            0.6
265_Yahoo_A4Benchmark-TS87_data_500_1616                           1.0""".split("\n")

In [ ]:
interesting_datasets = [x.split(" ")[0] for x in interesting]

In [ ]:
def load_series(files):
    results = {}
    for f in files:
        df = pd.read_csv("benchmark_exp/TSB-AD/TSB-AD-U/" + f + ".csv")
        results[f.split(".")[0]] = df
    return results

res = load_series(interesting_datasets)

In [ ]:
seasonal_decompose = pd.read_csv("results_seasonal_decompose.csv", index_col=0)
seasonal_decompose['algorithm'] = "seasonal decompose"
seasonal_decompose = seasonal_decompose.rename(columns={'name': 'dataset'})

In [ ]:
df = pd.read_parquet("dd2ab5d8-43f0-4a58-acf4-70285684f4be_0_49abff8e02de4e998b1ab9a30f3d7878.parquet").set_index("DataSetName")

def get_score_sd(dataset):
    score = df.loc[dataset, 'series_decompose_anomalies_Values_ad_score']
    return np.abs(pd.Series(score[1:-1].split(b",")).astype(float))

In [ ]:
def get_score(method, dataset):
    return np.load(f"benchmark_exp/score/uni/{method}/{d}.npy")

In [ ]:
from TSB_AD.utils.slidingWindows import find_length_rank
from utils import plot_detection
from sklearn.metrics import average_precision_score

for d in interesting_datasets:
    score_SR = get_score("SR", d)
    window_length = find_length_rank(res[d]['Data'])
    labels = res[d]['Label']
    plot_detection(res[d]['Data'], labels, scores=score_SR, window_length=window_length)
    plt.title(f"SR {d} : {average_precision_score(labels, score_SR):.2f}  ({window_length})")
    score_SD = get_score_sd(d)
    plot_detection(res[d]['Data'], labels, scores=score_SD, window_length=window_length)
    plt.title(f"Seasonal Decompose {d} :{average_precision_score(labels, score_SD):.2f} ({window_length})")